In [1]:
import warnings
warnings.filterwarnings('ignore')
from Model_test import *
from Load_and_Clean_data import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling
from sklearn.decomposition import PCA

In [2]:
pd.options.display.max_columns = 90

## Load, Clean and Save


In [ ]:
# Load and clean the data
df = load_raw_data()
df = clean_data(df)

# get features and target
X,y = df_featues_target(df)

# save to a file
save_X_y(X,y)


In [3]:
#load from file
X,y = load_X_y()

In [ ]:
# pandas_profiling.ProfileReport(df)

## Train Test Split

In [34]:
X_train, X_test, y_train, y_test = scale_train_test_split(X,y)

## Base Model

In [ ]:
models, model_names = train_model(X_train, y_train)

In [ ]:
y_train_predicts = predict_all(X_train, y_train, models)

In [ ]:
y_test_predicts = predict_all(X_test, y_test, models)

In [ ]:

print('--- Train data ---')
for i,y_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(y_train, y_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_test_pred in enumerate(y_test_predicts):
    display_acc_and_f1_score(y_test, y_test_pred, model_names[i])

In [ ]:
# Decision Tree overfitting because we did not limit number of splits yet

## Iteration #2

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

sm = SMOTE(random_state=109)
X_new_train, y_new_train = sm.fit_sample(X_train, y_train)



In [ ]:
# print(np.unique(y_new, return_counts=True))
# np.unique(y, return_counts=True)

In [ ]:
type(X_new_train), type(y_new_train)

In [ ]:
type(X_test), type(y_test)

In [ ]:
models, model_names = train_model(X_new_train, y_new_train)

In [ ]:
y_new_train_predicts = predict_all(X_new_train, y_new_train, models)

In [ ]:
y_new_test_predicts = predict_all(np.array(X_test), np.array(y_test), models)

In [ ]:

print('--- Train data ---')
for i, y_new_train_pred in enumerate(y_new_train_predicts):
    display_acc_and_f1_score(y_new_train, y_new_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_new_test_pred in enumerate(y_new_test_predicts):
    display_acc_and_f1_score(y_test, y_new_test_pred, model_names[i])

## Iteration #3

In [ ]:
len(X.columns)

In [4]:
from sklearn.decomposition import PCA
import numpy as np

pca_1 = PCA(n_components=20)
pca_2 = PCA(n_components=40)
pca_3 = PCA(n_components=60)

principalComponents = pca_1.fit_transform(X)
principalComponents = pca_2.fit_transform(X)
principalComponents = pca_3.fit_transform(X)

print(np.sum(pca_1.explained_variance_ratio_))
print(np.sum(pca_2.explained_variance_ratio_))
print(np.sum(pca_3.explained_variance_ratio_))





# print(pca_1.explained_variance_ratio_)
# print(pca_2.explained_variance_ratio_)
# print(pca_3.explained_variance_ratio_)
# print(np.sum(pca_2.explained_variance_ratio_))
# print(np.sum(pca_3.explained_variance_ratio_))

0.9975261370601384
0.9998231123255432
0.9999964313060081


In [5]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(X)
print(np.sum(pca.explained_variance_ratio_))

0.9267936605318506


In [6]:
pca_df = pd.DataFrame(principalComponents, columns= ['PCA1','PCA2','PCA3'])

In [7]:
X_train, X_test, y_train, y_test = scale_train_test_split(pca_df,y)

In [8]:
models, model_names = train_model(X_train, y_train)

In [9]:
y_train_predicts = predict_all(X_train, y_train, models)
y_test_predicts = predict_all(X_test, y_test, models)

In [10]:
print('--- Train data ---')
for i, y_new_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(y_train, y_new_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_new_test_pred in enumerate(y_test_predicts):
    display_acc_and_f1_score(y_test, y_new_test_pred, model_names[i])

--- Train data ---
Model: AdaBoost
Accuracy: 0.5412321157172056
F1-Score: 0.5412321157172056
Model: GradientBoost
Accuracy: 0.5439966458780986
F1-Score: 0.5439966458780986
Model: XGBoost
Accuracy: 0.5423064828887375
F1-Score: 0.5423064828887375
Model: Logistic Regresion
Accuracy: 0.42444054294848277
F1-Score: 0.4244405429484828
Model: DecisionTree_gini
Accuracy: 0.5463812169173523
F1-Score: 0.5463812169173523
Model: DecisionTree_entropy
Accuracy: 0.5457392170221687
F1-Score: 0.5457392170221687


--- Test data ---
Model: AdaBoost
Accuracy: 0.5404842386604827
F1-Score: 0.5404842386604827
Model: GradientBoost
Accuracy: 0.5420957471896863
F1-Score: 0.5420957471896863
Model: XGBoost
Accuracy: 0.5420957471896863
F1-Score: 0.5420957471896863
Model: Logistic Regresion
Accuracy: 0.4184419463878626
F1-Score: 0.4184419463878626
Model: DecisionTree_gini
Accuracy: 0.5372612216020753
F1-Score: 0.5372612216020753
Model: DecisionTree_entropy
Accuracy: 0.5372219165159972
F1-Score: 0.5372219165159972


## Iteration #4

In [ ]:
pipe_svm = Pipeline([('pca', PCA(n_components=27)),
            ('clf', svm.SVC(random_state=123))])

# Set grid search params
param_grid_svm = [
  {'clf__C': [0.1, 1, 10]  , 'clf__kernel': ['linear']},
  {'clf__C': [1, 10], 'clf__gamma': [0.001, 0.01], 'clf__kernel': ['rbf']},
 ]

# Construct grid search
gs_svm = GridSearchCV(estimator=pipe_svm,
            param_grid=param_grid_svm,
            scoring='accuracy',
            cv=3, verbose=2, return_train_score = True)

# Fit using grid search
gs_svm.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_svm.best_score_)

# Best params
print('\nBest params:\n', gs_svm.best_params_)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [9]:
lr = LogisticRegression(fit_intercept=False)
pipe = Pipeline([('logistic', lr)])
param_grid = [{'lr__penalty':['l1','l2'],
               'lr__C':[.01,.1,1.]
                }]
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, verbose=1)

In [10]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: Invalid parameter lr for estimator Pipeline(memory=None,
         steps=[('logistic',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=False, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='warn', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [16]:
lr.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [25]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import SelectKBest

In [26]:
# Create a combined preprocessing object
preprocess = FeatureUnion([('pca', PCA()), ("kbest", SelectKBest(k=1))])

In [27]:
# Create a pipeline
pipe = Pipeline([('preprocess', preprocess), ('classifier', LogisticRegression())])

In [28]:
# Create space of candidate values
search_space = [{'preprocess__pca__n_components': [1, 2, 3],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(0, 4, 10)}]

In [29]:
# Create grid search 
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)

In [30]:
# Fit grid search
best_model = clf.fit(X, y)

In [31]:
# View best hyperparameters
print('Best Number Of Princpal Components:', best_model.best_estimator_.get_params()['preprocess__pca__n_components'])
print('Best Penalty:', best_model.best_estimator_.get_params()['classifier__penalty'])
print('Best C:', best_model.best_estimator_.get_params()['classifier__C'])

Best Number Of Princpal Components: 3
Best Penalty: l1
Best C: 1.0


## Columns to inspect

    - Admission_type_id
  a  - discharge_disposition_id
    - admission_source_id
    - number_of_lab_procedures
    